<img src="valorumdata.svg" height="100px" width="200px" style="float: right;"/>

# Pandas: storage_formats
---

**Prerequisites**

- [Intro to DataFrames and Series](./intro.ipynb)

**Outcomes**

- Understand that data can be saved in various formats
- Know where to get help on file input and output
- Know when to use csv, xlsx, feather, and sql formats

---
    


In [2]:
# imports
import pandas as pd
import numpy as np

## File formats

Data can be saved to your computer in various formats. 

Pandas understands how to read and write DataFrames to many of these formats. 

We defer to the [official documentation](https://pandas.pydata.org/pandas-docs/stable/io.html) for a full description of how to interact with all the file formats, but will briefly discuss a few of them here.

### CSV

**What is it?** stores data as plain text (string) where each row is a line and columns are separated by `,`

**Pros**

- Widely used (you should be familiar with it)
- Plain text file (can open on any computer, "future proof")
- Can be read from and written to by (almost?) all data software


**Cons**

- Not the most efficient way to store or access

**When to use**:

- A great default option for most use cases

### xlsx

**What is it?** Binary file format used as the default when working in Excel.

**Pros**:

- Standard format in many industries
- Easy to share with colleagues that use Excel

**Cons**:

- Quite slow to read/write large amounts of data
- Stores both data and _metadata_ like styling and display information

**When to use**:

- When sharing data with Excel using colleagues
- When you would like special formatting to be applied to the spreadsheet when it is viewed in Excel

### Feather

**What is it?** Custom binary format designed for efficient reading and writing.

**Pros**:

- _Very_ fast

**Cons**: 

- Can only read and write from Python and a handful of other programming languages
- New file format (March '16), so most files don't come in this format

**When to use**:

- If you have "not small" amounts (> 100 MB) of unchanging data that you want to read quickly

### SQL

**What is it?** SQL is a language used to interact with relational databases... [more info](https://en.wikipedia.org/wiki/SQL)

**Pros**:

- Well established industry standard for handling data
- Most (don't quote me) of the world's data is in a SQL database somewhere

**Cons**:

- Complicated: to have full control you need to learn another language (SQL)

**When to use**:

- When reading from or writing to existing SQL databases.

**NOTE**: We cover interacting with SQL databases in a dedicated lecture

## Writing DataFrames

Let's now talk about saving a DataFrame to a file.

As a general rule of thumb, if we have a DataFrame `df` and we would like to save to save it as a file of type `FOO`, then we would call the method named `df.to_FOO(...)`.

We will show you how this can be done and try to highlight some of the items mentioned above as we go forward.

But, we will not cover all the possible options and features. We feel it is best to learn these as you need them by consulting the appropriate documentation.

First we need some DataFrames to save... the cell below makes some up.

Note that by default `df2` will be approximately 150 MB. 

If you need to change this number for any reason, change the value of the `wanted_mb` variable below

In [123]:
df1 = pd.DataFrame(
    np.random.randint(0, 100, size=(10, 4)), 
    columns=["a", "b", "c", "d"]
)

wanted_mb = 150  # CHANGE THIS LINE
nrow = 100000
ncol = int(((wanted_mb * 1024**2) / 8) / nrow)
df2 = pd.DataFrame(
    np.random.rand(nrow, ncol),
    columns=["x{}".format(i) for i in range(ncol)]
)

print("df2.shape = ", df2.shape)
print("df2 is approximately {} MB".format(df2.memory_usage().sum() / (1024**2)))

df2.shape =  (100000, 196)
df2 is approximately 149.5362091064453 MB


### [`df.to_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

Let's start with `df.to_csv`


Without any additional arguments, the `df.to_csv` function will return a string containing the csv form of the DataFrame:

In [99]:
print(df1.to_csv())

,a,b,c,d
0,7,42,46,85
1,57,73,56,29
2,84,64,64,96
3,71,59,81,90
4,8,53,90,67
5,75,8,89,20
6,78,98,87,49
7,26,39,57,93
8,22,0,88,75
9,47,49,0,46



If we do pass an argument, the first one will be used as the file name

In [124]:
df1.to_csv("df1.csv")

Run the cell below to verify that the file was created

In [107]:
import os
os.path.isfile("df1.csv")

True

Let's see how long it takes to save `df2` to a file (the `%%time` at the top will have Jupyter report the total time to run all the code in the cell)

In [108]:
%%time 
df2.to_csv("df2.csv")

CPU times: user 30.3 s, sys: 564 ms, total: 30.8 s
Wall time: 31.2 s


As we will see below, this isn't as fast as some other formats.

### [`df.to_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html)

When saving a DataFrame to an Excel workbook, we have the ability to choose both the name of the workbook (file) and the name of the sheet within the file where the DataFrame should be written.

We do this by passing the workbook name as the first argument and the sheet name as the second argument as follows

In [138]:
df1.to_excel("my_workbook.xlsx", "df1")

Pandas also gives us the option to write more than one DataFrame to a workbook

To do this we need to first construct an instance of `pd.ExcelWriter` and then pass that as the first argument to `df.to_excel`

Let's see how this works

In [139]:
with pd.ExcelWriter("df1.xlsx") as writer:
    df1.to_excel(writer, "df1")
    (df1 + 10).to_excel(writer, "df1 plus 10")

The `with .. as ... :` syntax used above is an example of a context manager

We don't need to understand all the details behind what this means (google it if you are curious)

For now just recognize that particular syntax as the way to write multiple sheets to an Excel workbook

<p style="color:red;">WARNING</p> Don't run the cell below unless you have a few minutes to spare

In [82]:
%%time
df2.to_excel("df2.xlsx")

CPU times: user 4min 54s, sys: 2.72 s, total: 4min 57s
Wall time: 4min 58s


### [`df.to_feather`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_feather.html)

As noted above, the feather file format was developed for very efficient reading and writing between Python and your computer

Support for this format is provided by a separate Python package called `feather-format`

This package is not installed by default. To install it, run the code cell below

In [49]:
!conda install -y -q feather-format -c conda-forge



Package plan for installation in environment /Users/sglyon/anaconda3:

The following NEW packages will be INSTALLED:

    arrow-cpp:      0.7.1-py36_2 conda-forge
    feather-format: 0.4.0-py36_2 conda-forge
    parquet-cpp:    1.3.0.post-2 conda-forge
    pyarrow:        0.7.1-py36_1 conda-forge



The only parameter for `df.to_feather` is the file name

Let's try it out

In [145]:
df1.to_feather("df1.feather")

In [109]:
%%time 
df2.to_feather("df2.feather")

CPU times: user 863 ms, sys: 122 ms, total: 985 ms
Wall time: 1.03 s


On my computer, I obtained the following timing results

| format | time |
| ------ | ---- |
| csv | 31.2 seconds |
| xlsx | 4 minutes 58 seconds | 
| feather | 1.03 seconds | 

As you can see, saving this DataFrame in the feather format was far faster than either csv or Excel

## Reading files into DataFrames

As with the `df.to_FOO` family of methods, there are similar `pd.read_FOO` functions (note they are in defined pandas, not as methods on a DataFrame)

Due to the various things that can be messy or wrong about data stored in files, these methods have many more options.

These will be explored in more detail in a separate module.

For now we just want to highlight the differences in how to read data from each of the file formats.

Let's start by reading in the files we just created and verify that they match the data we started out with

In [151]:
# notice that I specified that the index was in the first (0th -- why?) column of the file
df1_csv = pd.read_csv("df1.csv", index_col=0)
df1.equals(df1_csv)

True

In [152]:
df1_xlsx = pd.read_excel("my_workbook.xlsx", "df1", index_col=0)
df1.equals(df1_xlsx)

True

In [154]:
# notice feather already knows what the index is
df1_feather = pd.read_feather("df1.feather")
df1.equals(df1_feather)

True

With the `pd.read_FOO` family of functions, we can also read files from places on the internet.

I have saved the `df1` DataFrame we have been working with to a file and posted online

Below we show an example of using `pd.read_csv` to read this file

In [164]:
df1_url = "https://s3.us-east-2.amazonaws.com/valorum-materials/df1.csv"
df1_web = pd.read_csv(df1_url, index_col=0)
df1.equals(df1_web)

True

## Practice

Now it's your turn...

In the cell below, the variable `url` contains a web address to a csv file containing the result of all NFL games from September 1920 
to February 2017

Your task is to do the following:

- Use `pd.read_csv` to read this file into a DataFrame named `nfl`
- Print the shape and column names of `nfl`
- Save the DataFrame to a file named `nfl.xlsx`
- Open the spreadsheet using Excel on your computer

If you finish quickly, do some basic analysis of the data. Try to do something interesting. If you get stuck, here are some suggestions for what to try:

- Compute the average total points in each game (note, you will need to sum two of the columns to get total points)
- Repeat the above calculation, but only for playoff games 
- Compute the average score for your favorite team (you'll need to consider when they were team1 vs team2)
- Compute the ratio of "upsets" to total games played. An upset is defined as a team with a lower ELO winning the game

In [155]:
url = "https://raw.githubusercontent.com/fivethirtyeight/nfl-elo-game/"
url = url + "3488b7d0b46c5f6583679bc40fb3a42d729abd39/data/nfl_games.csv"

# your code here